In [1]:
from __future__ import division
# 避免整数相除为0(上面这个前面不能有注释)
import operator
import numpy as np

# 从util引入获取训练集的函数
import sys
sys.path.append(r"F:\\FictionRecommend\\")
import Ipynb_importer
import process as read
import test as mat_util
import numpy as np
# 求矩阵的逆
from scipy.sparse.linalg import gmres

importing Jupyter notebook from process.ipynb


In [2]:
# 测试文件
log_file = "F:/badou/tmp/data/recommender/personalRank/log.txt"
# 评分文件
rating_file = "F:/badou/tmp/data/recommender/personalRank/ratings.csv"
# 用来看推荐效果
movies_file = "F:\\badou\\tmp\\data\\recommender\\data\\ml-latest-small\\movies.csv"

In [3]:
def personal_rank(graph, root, alpha, iter_num, recom_num = 10):
    """
    基础算法版本
    :param graph:
        根据用户的行为得到的用户和item的图结构
    :param root:
        将要给那个user推荐 root顶点
    :param alpha:
        以 alpha 的概率选择向下游走，以 1 - alpha 的概率选择回到起点
    :param iter_num:
        迭代次数
    :param recom_num:
        推荐个数(默认10个)
    :return:
        dict 长度为推荐个数 recom_num
            key -> item_id
            value -> pr值(personal rank)
    """
    # 存储所有的顶点对于root顶点的personal_rank值
    rank = {}
    # 初始值，除root顶点以外其余顶点personal_rank值都为0
    rank = {point: 0 for point in graph}
    # root顶点初始化为1
    rank[root] = 1
    # 输出的推荐dict
    recom_result = {}
    
    for iter_index in range(iter_num):
        # 用来存储当前迭代轮次下其余顶点对root顶点的pr值
        tmp_rank = {point: 0 for point in graph}
        
        # 上部分公式
        # 如果该顶点不是root顶点，则它的pr值 = 所有连接到该顶点的顶点将自己的pr值以 1/n 的概率贡献到该顶点上，
        # 那么n就是连接到该顶点的这个顶点的出度
        for out_point, out_dict in graph.items():
            for inner_point, value in graph[out_point].items():
                tmp_rank[inner_point] += round(alpha * rank[out_point] / len(out_dict), 4)
                # 下半部分公式：如果该顶点同时是root顶点的话，不仅有公式上半部分构成还要加上 1 - alpha
                if inner_point == root:
                    tmp_rank[inner_point] += round(1 - alpha, 4)
        
        # 完全相同时可以停止迭代(收敛)
        if tmp_rank == rank:
            break
        
        # 继续往下迭代
        rank = tmp_rank
    
    right_num = 0
    # 按pr值的得分进行排序，并过滤掉其中的user顶点和root顶点已经产生行为的item，从而得到推荐结果
    for zuhe in sorted(rank.items(), key = operator.itemgetter(1), reverse  = True):
        point, pr_score = zuhe[0], zuhe[1]
        # 如果顶点不是item顶点，需要过滤掉
        if len(point.split("_")) < 2:
            continue
        # 如果顶点是item顶点，但是和root顶点产生过行为，同样过滤掉
        if point in graph[root]:
            continue
            
        recom_result[point] = pr_score
        right_num  += 1
        if right_num > recom_num:
            break
    return recom_result

In [4]:
def get_one_user_recom():
    '''
    用户的推荐列表
    :return:
    '''
    user = "A"
    # user = "1"
    alpha = 0.6
    iter_num  = 100
    
    
    # {'item_c': 0.0726, 'item_e': 0.0363} 测试，明显 c > e
    graph = read.get_graph_from_data(log_file)
    recom_result = personal_rank(graph, user, alpha, iter_num)
    
    return recom_result

In [5]:
recom_result_base = get_one_user_recom()
print(recom_result_base)

{'item_c': 0.074, 'item_e': 0.037}
